In [1]:
import pandas as pd

# Load the provided Excel file
file_path = '/content/sample_data/다중회귀_복합SCFI_2017.xlsx'
data = pd.read_excel(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()

,YYYYMM,CH_EXP,FLEET_DEV,CNTR_TIMECHART,WTI,R_GDP_WORLD,DEFLATOR_WORLD,R_GDP_USA,CLI_USA,PMI_US,DEFLATOR_USA,G_RELI,G_DELAY,SCFI_C
0,2014-01,21790540,17312194,46.79983,94.6,1.787676e+07,109.776667,4173.316667,100.7090,53.7,102.690000,0.694882,3.948304,1174.866
1,2014-02,9977430,17385239,46.71077,100.8,1.791018e+07,110.063333,4168.433333,100.7202,57.1,102.830000,0.681136,4.267981,1069.362
2,2014-03,17115050,17367549,46.70015,100.8,1.794359e+07,110.350000,4163.550000,100.7613,55.5,102.970000,0.721154,3.897523,969.145
3,2014-04,20089670,17534040,47.20250,102.1,1.799153e+07,110.543333,4181.376667,100.8148,55.4,103.163333,0.732795,3.648607,1078.462
4,2014-05,20500550,17701139,46.95479,102.2,1.803947e+07,110.736667,4199.203333,100.8649,56.4,103.356667,0.753047,3.736327,1137.112


# **도표 5-25. 원-핫 인코딩 파이썬 코드**

In [2]:
# data['YYYYMM'] 컬럼을 datetime 객체로 변환하고, 변환된 결과의 월(month)만 추출하여 새로운 컬럼 'Month'를 생성합니다.
data['Month'] = pd.to_datetime(data['YYYYMM'], format='%Y-%m').dt.month

# 원-핫 인코딩을 적용하여 각 월을 나타내는 이진 특성(컬럼)을 생성합니다.
# 'Month' 컬럼의 데이터를 get_dummies 함수를 사용하여 인코딩하고, prefix='Month'를 사용하여 새로운 컬럼 이름을 'Month_월' 형식으로 설정합니다.
month_one_hot = pd.get_dummies(data['Month'], prefix='Month')

# 원-핫 인코딩된 DataFrame에서 첫 번째 열(1월을 나타내는 열)을 제외합니다.
# 이는 더미 변수 함정을 피하기 위한 일반적인 방법으로, 모델에 모든 월을 포함시키면 다중공선성 문제가 발생할 수 있기 때문입니다.
month_one_hot = month_one_hot.iloc[:, 1:]

# 원래의 'YYYYMM' 컬럼과 원-핫 인코딩된 월 데이터를 가로로 합칩니다.
data_with_only_month_one_hot = pd.concat([data['YYYYMM'], month_one_hot], axis=1)

# 결과 데이터 프레임의 처음 5개 행을 출력하여, 데이터가 올바르게 처리되었는지 확인합니다.
data_with_only_month_one_hot.head()

,YYYYMM,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12
0,2014-01,False,False,False,False,False,False,False,False,False,False,False
1,2014-02,True,False,False,False,False,False,False,False,False,False,False
2,2014-03,False,True,False,False,False,False,False,False,False,False,False
3,2014-04,False,False,True,False,False,False,False,False,False,False,False
4,2014-05,False,False,False,True,False,False,False,False,False,False,False


# 도표 5-26. 초기 모형과 명목변수를 추가한 다중회귀모형의 평가지표 비교

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm


# YYYYMM을 datetime으로 변환 후 월 추출
data['Month'] = pd.to_datetime(data['YYYYMM'], format='%Y-%m').dt.month

# 원-핫 인코딩 (1월 제외)
month_one_hot = pd.get_dummies(data['Month'], prefix='Month').iloc[:, 1:]

# 원-핫 인코딩된 데이터와 기존 데이터 병합
data = pd.concat([data, month_one_hot], axis=1)

# 종속 변수와 독립 변수 분리
X = data.drop(['YYYYMM', 'Month', 'SCFI_C'], axis=1)
y = data['SCFI_C']

# 데이터 타입 확인 및 변환
X = X.astype(float)

# 훈련 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# scikit-learn을 사용한 회귀 분석
model = LinearRegression()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# R-squared 계산
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Adjusted R-squared 계산 함수
def adjusted_r2(r_squared, n, p):
    return 1 - (1 - r_squared) * (n - 1) / (n - p - 1)

# 샘플 수와 독립 변수의 수 계산
n_train = len(y_train)
n_test = len(y_test)
p = X_train.shape[1]

# scikit-learn을 사용한 조정된 결정계수 계산
adj_r2_train = adjusted_r2(r2_train, n_train, p)
adj_r2_test = adjusted_r2(r2_test, n_test, p)

# statsmodels를 사용한 회귀 분석
X_train_sm = sm.add_constant(X_train)
X_test_sm = sm.add_constant(X_test)
model_sm = sm.OLS(y_train, X_train_sm).fit()

# statsmodels를 사용한 조정된 결정계수 계산
adj_r2_train_sm = model_sm.rsquared_adj
y_test_pred_sm = model_sm.predict(X_test_sm)
r2_test_sm = r2_score(y_test, y_test_pred_sm)
adj_r2_test_sm = adjusted_r2(r2_test_sm, n_test, p)

# 결과 출력
results_sm = pd.DataFrame({
    '훈련 데이터': [model_sm.rsquared, adj_r2_train_sm],
    '테스트 데이터': [r2_test_sm, adj_r2_test_sm]
}, index=['결정계수', '조정된 결정계수'])

results_sm

,훈련 데이터,테스트 데이터
결정계수,0.977188,0.918182
조정된 결정계수,0.902235,1.106363


# 도표 5-27. 단순선형회귀 ChatGPT 요청과 sm.OLS( ) 파이썬 코드

In [5]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

# 파일 경로
file_path = '/content/sample_data/단순회귀_복합SCFI_2017.xlsx'

# 엑셀 파일 불러오기
df = pd.read_excel(file_path)

# 독립변수와 종속변수 설정
X = df['CLI_USA']
y = df['SCFI_C']

# 상수항 추가
X = sm.add_constant(X)

# OLS 모델 적합
model = sm.OLS(y, X).fit()

# 잔차 계산
residuals = model.resid

# 잔차 제곱합 (SSR)
SSR = np.sum(residuals ** 2)

# 잔차의 자유도
n = len(y)
k = X.shape[1] - 1
df_residuals = n - k - 1

# 잔차의 분산
variance_of_residuals = SSR / df_residuals

# 잔차의 표준오차
standard_error_of_regression = np.sqrt(variance_of_residuals)

# 결과 출력
print("잔차의 표준편차: ", standard_error_of_regression)
# 모델 요약 출력
model_summary = model.summary()
model_summary

잔차의 표준편차:  105.48522822654937


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 SCFI_C   R-squared:                       0.737
Model:                            OLS   Adj. R-squared:                  0.730
Method:                 Least Squares   F-statistic:                     120.3
Date:                Sun, 19 May 2024   Prob (F-statistic):           4.88e-14
Time:                        07:47:30   Log-Likelihood:                -272.47
No. Observations:                  45   AIC:                             548.9
Df Residuals:                      43   BIC:                             552.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.504e+04   2358.018    -10.617      0.000   -2.98e+04   -2.03e+04
CLI_USA      258.6228     23.583     10.966      0.000     211.063     306.183
==============================================================================
Omnibus:                        0.501   Durbin-Watson:                   0.610
Prob(Omnibus):                  0.778   Jarque-Bera (JB):                0.600
Skew:                          -0.223   Prob(JB):                        0.741
Kurtosis:                       2.653   Cond. No.                     1.50e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.5e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:
# 잔차의 표준편차를 잘못계산하는 경우

print(" model.resid.std() : ",  model.resid.std()) # 104.27964316751095    # Calculate the standard error of the residuals (residual standard error)
print(" 잔차제곱합/잔차자유도 의 제곱근 : ", np.sqrt((model.resid ** 2).sum() / model.df_resid)) # 105.48522822654941

 model.resid.std() :  104.27964316751094
 잔차제곱합/잔차자유도 의 제곱근 :  105.48522822654937
